# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [28]:
#!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [38]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [39]:
partone_data = pd.read_csv("output_data/city_data.csv", dtype="object")
partone_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,lompoc,34.64,-120.46,69.01,64,10,10.29,US,1595622657
1,banjar,-8.19,114.97,77.0,78,40,13.87,ID,1595622658
2,hermanus,-34.42,19.23,50.0,84,0,1.92,ZA,1595622658
3,rio grande,-32.03,-52.1,55.4,87,90,18.34,BR,1595622480
4,seoul,37.57,126.98,73.4,69,90,4.7,KR,1595622600


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [40]:
# Configure gmaps with API key
gmaps.configure(api_key= g_key)

In [41]:
# Store 'Lat' and 'Lng' into  locations 
locations = partone_data[["Lat", "Lng"]].astype(float)

# Convert Humidity to float and store
humidity = partone_data["Humidity"].astype(float)

In [42]:
# Create a humididty Heatmap layer
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights= humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [43]:
partone_data["Max Temp"] = pd.to_numeric(partone_data["Max Temp"])
partone_data["Wind Speed"] = pd.to_numeric(partone_data["Wind Speed"])
partone_data["Cloudiness"] = pd.to_numeric(partone_data["Cloudiness"])

In [44]:
narrowed = partone_data.loc[(partone_data["Max Temp"] < 77) & (partone_data["Max Temp"] > 70) & (partone_data["Wind Speed"] < 10)  & (partone_data["Cloudiness"] == 0), :].dropna()
narrowed_cities = narrowed.reset_index()
del narrowed_cities["index"]
del narrowed_cities["Humidity"]
del narrowed_cities["Cloudiness"]
del narrowed_cities["Wind Speed"]
del narrowed_cities["Date"]
narrowed_cities

,City,Lat,Lng,Max Temp,Country
0,ponta do sol,32.67,-17.1,73.00,PT
1,aksu,41.12,80.26,76.23,CN
2,eldorado,-23.79,-54.28,75.45,BR
3,saint-francois,46.42,3.91,71.60,FR
4,popondetta,-8.75,148.25,72.37,PG
5,ilhabela,-23.78,-45.36,70.47,BR
6,shymkent,42.3,69.6,75.20,KZ


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [45]:
hotels = []

for i in range(len(narrowed_cities)):
    lat = narrowed_cities.loc[i]["Lat"]
    lng = narrowed_cities.loc[i]["Lng"]

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    
    try:
        hotels.append(response['results'][0]['name'])
    except:
        hotels.append("")
        
narrowed_cities["Hotel Name"] = hotels
del narrowed_cities["Max Temp"]
hotel_df = narrowed_cities.dropna()
hotel_df

,City,Lat,Lng,Country,Hotel Name
0,ponta do sol,32.67,-17.1,PT,Ribeira Brava
1,aksu,41.12,80.26,CN,Aksu Prefecture
2,eldorado,-23.79,-54.28,BR,Eldorado
3,saint-francois,46.42,3.91,FR,Molinet
4,popondetta,-8.75,148.25,PG,Popondetta
5,ilhabela,-23.78,-45.36,BR,Ilhabela
6,shymkent,42.3,69.6,KZ,Shymkent


In [46]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]].astype(float)

In [47]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))